# Train a JointVAE model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import torch
from viz.visualize import Visualizer
use_cuda = torch.cuda.is_available()
use_cuda

True

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # change to your device

#### Prepare data list

In [4]:
!ls data

dress_dresslen_train_test_splits.json	dress_sleeve_train_test_splits.json
dress_sleevelen_train_test_splits.json	loadable_women_primary_dress.csv


#### Create list of image paths

In [5]:
loadable_dresses = list(np.loadtxt('data/loadable_women_primary_dress.csv',delimiter=',',skiprows=1,dtype='str'))

In [6]:
print(len(loadable_dresses))

102125


In [7]:
loadable_dresses[:5]

['/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/4/6418008_9882769.jpg',
 '/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/6/6627534_9864695.jpg',
 '/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/7/6772508_9949243.jpg',
 '/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/7/6758001_9588597.jpg',
 '/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/6/6637527_9387666.jpg']

In [8]:
bad_data = ['/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/7/2/723739_1342692.jpg']
for i, bad in enumerate(bad_data):
    if bad in loadable_dresses:
        del loadable_dresses[i]
        
print(len(loadable_dresses))

102124


#### Split into train and test set

In [9]:
#image_paths_train = loadable_dresses[:int(len(loadable_dresses)*0.87)]
#image_paths_test = loadable_dresses[int(len(loadable_dresses)*0.87):]
image_paths_train = loadable_dresses[:88800]
image_paths_test = loadable_dresses[88800:-124]


print(f"Number of train image paths: {len(image_paths_train):,d}")
print(f"Number of test image paths: {len(image_paths_test):,d}")
print()
print("Sample paths:")
print(image_paths_train[0])
print(image_paths_train[-1])
print(image_paths_test[0])
print(image_paths_test[-1])

Number of train image paths: 88,800
Number of test image paths: 13,200

Sample paths:
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/6/6/6627534_9864695.jpg
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/5/2/529420_884192.jpg
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/5/2/527981_1034607.jpg
/home/jovyan/vmldata/raw_source_data/v20180810_all_wearables/3/4/341494_806784.jpg


In [10]:
#from utils.dataloaders import get_mnist_dataloaders, get_fashion_mnist_dataloaders
#train_loader, test_loader = get_mnist_dataloaders(batch_size=64)
#train_loader, test_loader = get_fashion_mnist_dataloaders(batch_size=64)

In [12]:
from torchvision import transforms
#from utils.dataloaders_custom import get_imagelist_dataloader, ImageListDataset
from utils.dataloader_tools import get_imagelist_dataloader, ImageListDataset

BATCH_SIZE = 200

composed = transforms.Compose([transforms.CenterCrop((90,90)),transforms.Resize((64,64)),transforms.ToTensor()])

train_dataset = ImageListDataset(image_paths_train, cut_from='top', cut_amount=90, transform=composed)
test_dataset = ImageListDataset(image_paths_test, cut_from='top', cut_amount=90, transform=composed)

train_loader = get_imagelist_dataloader(batch_size=BATCH_SIZE, dataset_object=train_dataset)
test_loader = get_imagelist_dataloader(batch_size=BATCH_SIZE, dataset_object=test_dataset)

### Define latent distribution of the model

In [13]:
# Latent distribution will be joint distribution of 10 gaussian normal distributions
# and one 10 dimensional Gumbel Softmax distribution
latent_spec = {'cont': 20,
               'disc': [20]}

### Build a model

In [20]:
!ls jointvae

decoder_scratchpad.py  models_128_v1.py  models_128_v4.py     models_v1.py
encoder_scratchpad.py  models_128_v2.py  models_64_xstyle.py  __pycache__
__init__.py	       models_128_v3.py  models.py	      training.py


In [36]:
#from jointvae.models_v1 import VAE
from jointvae.models_64_xstyle import VAE, Encoder_x, Decoder_x

#model = VAE(latent_spec=latent_spec, img_size=(3, 260, 260), use_cuda=use_cuda)
model = VAE(encoder=Encoder_x, decoder=Decoder_x, latent_spec=latent_spec, img_size=(3, 64, 64), use_cuda=use_cuda)

In [37]:
#print(model)

### Train the model

In [38]:
from torch import optim

# Build optimizer
optimizer = optim.Adam(model.parameters(), lr=3e-5, amsgrad=True) # added amsgrad # orig lr 5e-4

In [39]:
from jointvae.training import Trainer

# Define the capacities
# Continuous channels
cont_capacity = [0.0, 5.0, 25000, 30.0]  # Starting at a capacity of 0.0, increase this to 5.0
                                         # over 25000 iterations with a gamma of 30.0
# Discrete channels
disc_capacity = [0.0, 5.0, 25000, 30.0]  # Starting at a capacity of 0.0, increase this to 5.0
                                         # over 25000 iterations with a gamma of 30.0

# Build a trainer
trainer = Trainer(model, optimizer,
                  cont_capacity=cont_capacity,
                  disc_capacity=disc_capacity,
                 use_cuda=use_cuda)

#### Initialize visualizer

In [40]:
# Build a visualizer which will be passed to trainer to visualize progress during training
viz = Visualizer(model)

In [41]:
# Train model for 10 epochs
# Note this should really be a 100 epochs and trained on a GPU, but this is just to demo

trainer.train(train_loader, epochs=10, save_training_gif=None)

RuntimeError: bool value of Tensor with more than one value is ambiguous

In [ ]:
print('hi')

### Visualize

In [ ]:
# Plot reconstructions
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Get a batch of data
for batch, labels in test_loader:
    break

# Reconstruct data using Joint-VAE model
recon = viz.reconstructions(batch)

plt.figure(figsize=(26,26))
#np.transpose(recon.numpy(), (2,1,0))
plt.imshow(np.rot90(np.transpose(recon.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30_reconstructions.png",dpi=200)

In [ ]:
# Plot samples
samples = viz.samples()

plt.figure(figsize=(26,26))
#plt.imshow(samples.numpy()[0, :, :],cmap='gray');
plt.imshow(np.rot90(np.transpose(samples.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30.png",dpi=200)

#### Traverses all latent dimensions one by one and plots a grid of images where each row corresponds to a latent traversal of one latent dimension

In [ ]:
# Plot all traversals
traversals = viz.all_latent_traversals(size=20)

plt.figure(figsize=(26,26))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30_all_traversals_n20.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=2, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(20,20))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30.png_traversals2100.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=1, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(16,16))
plt.imshow(traversals.numpy()[0, :, :]);
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30_traversals1100.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=9, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(16,16))
#plt.imshow(traversals.numpy()[0, :, :], cmap='gray');
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/306/64/croptotop_64x64_e130_b200_c20d20_gam30_traversals9100.png",dpi=200)

In [ ]:
!ls

### Save Model

In [28]:
model_name = "croptotop_64x64_e130_b200_c20d20_gam30.pth"

In [29]:
torch.save(model.state_dict(),"trained_models" + "statedict_" + model_name) # save state dict
torch.save(model, model_name) # save full model

In [30]:
print("Done training: ",model_name)

Done training:  realdata64x64_e60_b512_cd40_gam20.pth


#### Restore Model from State Dict

In [28]:
sd_model = VAE(latent_spec=latent_spec, img_size=(3, 64, 64))
sd_model.load_state_dict(torch.load("statedict_" + model_name))

#### Restore Full Model
* Note in this case the serialized data is bound to the specific classes and exact directory strucutre used.

In [ ]:
full_model = torch.load(model_name)

In [ ]:
type(full_model)

In [ ]:
type(sd_model)

In [36]:
!ls

data				requirements.txt
dataloading_pytorch_test.ipynb	statedict_jvae_fmnist_oct292018.pth
imgs				trained_models
jointvae			training.gif
jvae_fmnist_oct292018.pth	training_rd1.gif
latent_traversals.py		train_model.ipynb
load_model.ipynb		train_model_realdata_306_260_v1.ipynb
main.py				train_model_realdata_64_v1.ipynb
__pycache__			utils
RandomUtilsandTests.ipynb	viz
README.md
